# My DE homework about crawling information of movies in some movies websites

### Step 1: create a connection to a DBSM

### Step 2: create a table on DBSM (here i use postgres in Dbeaver)

### Step 3: crawl data (information) of movies from the website and generate data

### Step 4: insert data into database

In [1]:
import bs4
import lxml
event = {
    "urls" : [
        "https://phimmoiyyy.net/the-loai/phim-chieu-rap",
        "https://phimmoiiii.net/the-loai/phim-hinh-su",
        "https://phimmoiiii.net/the-loai/phim-netflix"
    ],
    "table_name": "movies",
    "data_field": [
        {
            "field": "id",
            "type": "integer"
        },
        {
            "field": "movie_link",
            "type": "varchar"
        },
        {
            "field": "poster_image_link",
            "type": "varchar"
        },
        {
            "field": "movie_name",
            "type": "varchar"
        },
        {
            "field": "movie_status",
            "type": "varchar"
        },
        {
            "field": "en_name",
            "type": "varchar"
        },
        {
            "field": "vn_name",
            "type": "varchar"
        },
        {
            "field": "category",
            "type": "varchar"
        }
    ]
}

In [2]:
# Step 1: create a connection to DBSM

import psycopg2
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def create_connection():
    connection = psycopg2.connect(
        host="introduction-01-intro-ap-southeast-1-dev-introduction-db.cpfm8ml2cxp2.ap-southeast-1.rds.amazonaws.com",
        database="postgres",
        user="postgres",
        password="postgres123",
        port="5432"
    )
    connection.set_session(autocommit=True)
    return connection

connection = create_connection()
logger.info(f"connection log: {connection}")
    

INFO:root:connection log: <connection object at 0x103e77eb0; dsn: 'user=postgres password=xxx dbname=postgres host=introduction-01-intro-ap-southeast-1-dev-introduction-db.cpfm8ml2cxp2.ap-southeast-1.rds.amazonaws.com port=5432', closed: 0>


In [3]:

# Step 2: create a table on DBSM

def create_table(connection, table_name, fields):
    logger.info(f"Start create table {table_name}")
    list_field_name_field_type = [f'{field["field"]} {field["type"]}' for field in fields] 
    result_field_name_field_type = ','.join(list_field_name_field_type)

    create_table_str = f"""
        create table if not exists {table_name} (
            {result_field_name_field_type}
        )
    """

    cursor = connection.cursor()
    cursor.execute(create_table_str)


table_name = "ai4e_movie_minhtanpham"
fields = event["data_field"]

create_table(connection, table_name, fields)

INFO:root:Start create table ai4e_movie_minhtanpham


In [4]:
# Step 3: crawl data from the websites and generate

import requests
import bs4
import lxml

def crawl_data(event, total_movies):
    for url in event["urls"]:
        info = requests.get(url, headers={})

        soup = bs4.BeautifulSoup(info.content, features='html.parser')

        movies = soup.find_all("article", {"class": "item movies"})
        print(len(movies))

        for movie in movies:
            movies_id = movie["id"].split("-")[1]
            movie_footer = movie.find('div', {"class": "data"})
            movie_link = movie_footer.find("a")["href"]
            vn_name = movie_footer.find("a").text
            en_name = movie_footer.find("span").text
            poster_image_link = movie.find("img")["src"]
            movie_name = movie.find("img")["alt"]
            movie_status = movie.find('div', {"class": "trangthai"}).text
            category = "phim-chieu-rap"

            movie_dict = {
                "id": movies_id,
                "movie_link": movie_link,
                "poster_image_link": poster_image_link,
                "movie_name": movie_name,
                "movie_status": movie_status,
                "en_name": en_name,
                "vn_name": vn_name,
                "category": category
            }
            total_movies.append(movie_dict)
            
total_movies = []
crawl_data(event, total_movies)
print(len(total_movies))

/Users/tanpham/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


29
24
19
72


In [5]:
# Step 4: insert data into DBSM

def insert_database(connection, table_name, data_movies):
    logger.info(f"Start insert data into table {table_name}")
    list_field_name = [f'{field["field"]}' for field in event["data_field"]] 
    result_field_name = ','.join(list_field_name)

    data_insert = [f"({item['id']},'{item['movie_link']}','{item['poster_image_link']}','{item['movie_name']}','{item['movie_status']}','{item['en_name']}','{item['vn_name']}','{item['category']}')" for item in data_movies]

    result_insert = ','.join(data_insert)

    sql_insert = f"""
        INSERT INTO {table_name} ({result_field_name})
        VALUES {result_insert};
    """
    print(sql_insert)
    cursor = connection.cursor()
    cursor.execute(sql_insert)

    

insert_database(connection, table_name, total_movies)

INFO:root:Start insert data into table ai4e_movie_minhtanpham

        INSERT INTO ai4e_movie_minhtanpham (id,movie_link,poster_image_link,movie_name,movie_status,en_name,vn_name,category)
        VALUES (16155,'https://phimmoiiii.net/phim-le/bua-hinh-nhan','https://phimmoiiii.net/wp-content/uploads/2023/04/Bua-Hinh-Nhan.jpg','Bùa Hình Nhân','HD Vietsub',' Hoon Payon 2023 ','Bùa Hình Nhân','phim-chieu-rap'),(34569,'https://phimmoiiii.net/phim-le/hon-ma-khong-dau','https://phimmoiiii.net/wp-content/uploads/2022/07/hon-ma-khong-dau.jpg','Hồn Ma Không Đầu','HD Vietsub',' Ivanna 2022 ','Hồn Ma Không Đầu','phim-chieu-rap'),(3208,'https://phimmoiiii.net/phim-le/ma-lai-rut-ruot','https://phimmoiiii.net/wp-content/uploads/2023/04/Phim-Ma-Lai-Rut-Ruot.jpg','Ma Lai Rút Ruột','HD Vietsub',' Inhuman Kiss 2 2023 ','Ma Lai Rút Ruột','phim-chieu-rap'),(31855,'https://phimmoiiii.net/phim-le/ma-so-truc-quy','https://phimmoiiii.net/wp-content/uploads/2022/11/ma-so-truc-quy.jpg','Ma Sơ Trục Quỷ','Coming 